#Download and Import necessary modules

In [1]:
from openai import OpenAI

In [39]:
!pip install --quiet imapclient

#Load your credentials

In [49]:
#Store your secrets(credentials) in Google Secret Manager
from google.colab import auth, userdata

auth.authenticate_user()
Username = userdata.get("EMAIL")
PASSWORDD = userdata.get("PASSWORD")
API_KEY=userdata.get("OPENROUTER_API_KEY")


#Get data from your mail and then processing it

##Using imapClient to fetch mails from Gmail

In [50]:
import imapclient
import email
from email.header import decode_header

# Email credentials
EMAIL = Username
PASSWORD = PASSWORDD
IMAP_SERVER = "imap.gmail.com"

def fetch_emails():
    with imapclient.IMAPClient(IMAP_SERVER) as mail:
        mail.login(EMAIL, PASSWORD)
        mail.select_folder("INBOX")

        # Fetching unread emails
        messages = mail.search(["UNSEEN"])
        if not messages:
            print("No new emails.")
            return []

        # Fetch latest 5 unread emails
        response = mail.fetch(messages[-5:], ["RFC822"])

        emails = []
        for msgid, data in response.items():
            raw_email = data[b"RFC822"]
            msg = email.message_from_bytes(raw_email)

            # Decode subject
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding or "utf-8")

            sender = msg.get("From")

            # Extract email body
            body = ""
            if msg.is_multipart():
                for part in msg.walk():
                    content_type = part.get_content_type()
                    if content_type == "text/plain":
                        body = part.get_payload(decode=True).decode(errors="ignore")
                        break
            else:
                body = msg.get_payload(decode=True).decode(errors="ignore")

            emails.append({"sender": sender, "subject": subject, "body": body})

    return emails

emails = fetch_emails()

# Save emails to a file for the next script
import json
with open("emails.json", "w") as f:
    json.dump(emails, f)

print(f"Fetched {len(emails)} emails. Saved to 'emails.json'.")


Fetched 5 emails. Saved to 'emails.json'.


##AI analysis of the mail

In [52]:
import requests
import json

OPENROUTER_API_KEY = API_KEY

def analyze_email(subject, body):
    prompt = f"""
    You are an AI assistant helping manage emails. Analyze the following email and return:
    - Urgency level: "Urgent", "Important", or "Ignore" give answer in a sentence .
    - Emotional tone: "Angry", "Neutral", "Friendly" give answer in a sentence.
    - Suggested reply First choose whether to reply to the email or not. only if you choose to reply : (3-5 sentences) in a formal way. Write personalized mails ,Including regards and my Name "Pranav Sasank"
    Email:
    Subject: {subject}
    Body: {body}

    Return in this format:
    Urgency: [Urgency level]
    Tone: [Emotional tone]
    Reply: [Suggested reply]
    """

    headers = {"Authorization": f"Bearer {OPENROUTER_API_KEY}", "Content-Type": "application/json"}
    data = {
        "model": "deepseek/deepseek-chat:free",
        "messages": [{"role": "user", "content": prompt}],
    }

    response = requests.post("https://openrouter.ai/api/v1/chat/completions", json=data, headers=headers)
    return response.json()["choices"][0]["message"]["content"]


#Output

In [53]:
# Load emails from the file saved in the first script
with open("emails.json", "r") as f:
    emails = json.load(f)

# Analyze and print results
for email in emails:
    print(f"Analyzing email from {email['sender']} - Subject: {email['subject']}")
    analysis = analyze_email(email["subject"], email["body"])
    print(f"Analysis:\n{analysis}\n")

Analyzing email from Hugging Face Team <thomas.simonini@huggingface.co> - Subject: 🤖 Agents Course: New Unit on LangGraph library!
Analysis:
Urgency: Ignore  
Tone: Friendly  
Reply: No reply needed. This email is informational and does not require a response. Continue with the course at your own pace.

Analyzing email from "Indastro -  Pt. Punarvasu" <ptpunarvasu@indastro.com> - Subject: 6 Planets on 30th March 2025: Major Disruption Expected
Analysis:
Urgency: Ignore  
Tone: Neutral  
Reply: No reply needed. This email appears to be an astrological update and promotional content, which does not require immediate attention or response.

Analyzing email from LinkedIn <messages-noreply@linkedin.com> - Subject: Welcome to Premium, Pranav Sasank! Let’s get started
Analysis:
Urgency: Ignore  
Tone: Friendly  
Reply: No reply needed.

Analyzing email from Yahvi Khamitkar via LinkedIn <messaging-digest-noreply@linkedin.com> - Subject: Yahvi just messaged you
Analysis:
Urgency: Important  
To